In [2]:
lst = [1, 2, 3]
lst.append(lst)
lst

[1, 2, 3, [...]]

In [3]:
del lst

In [4]:
import sys

In [5]:
lst = [1, 2, 3]

sys.getrefcount(lst)

2

In [14]:
x = 200
x -= 200

sys.getrefcount(x)

5556

In [15]:
x = 200
x += 57

sys.getrefcount(x)

2

In [16]:
s1 = "abc"

sys.getrefcount(s1)

54

In [17]:
s2 = "abc"

In [18]:
s1 is s2

True

In [19]:
s3 = "!abc"

In [20]:
s4 = "!abc"

In [21]:
s3 is s4

False

In [28]:
lst = [1, 2, 3]
print("init", sys.getrefcount(lst))

tp = (5, lst)
print("tuple", sys.getrefcount(lst))

lst1 = lst2 = lst
print("lists", sys.getrefcount(lst))

del tp
print("del tp", sys.getrefcount(lst))

del lst1, lst2
print("del lists", sys.getrefcount(lst))


def count_refs(obj):
    print("count_refs", sys.getrefcount(obj))

    
count_refs(lst)
print("after count_refs", sys.getrefcount(lst))


lst.append(lst)
print("cycle", sys.getrefcount(lst))

init 2
tuple 3
lists 5
del tp 4
del lists 2
count_refs 4
after count_refs 2
cycle 3


In [32]:
lst

[1, 2, 3, [...]]

In [33]:
sys.getsizeof(lst), sys.getsizeof([]), sys.getsizeof([1]), sys.getsizeof([2 ** 100])

(120, 56, 64, 64)

In [34]:
sys.getsizeof(2 ** 100)

40

In [39]:
d = {}

sys.getsizeof(d), sys.getsizeof({1: 11})

(64, 232)

In [43]:
sys.getsizeof(""), sys.getsizeof("a"), sys.getsizeof("av"), sys.getsizeof("Щ")

(49, 50, 51, 76)

In [51]:
import ctypes
import gc

In [53]:
class PyObject(ctypes.Structure):
    _fields_ = [("refcnt", ctypes.c_long)]


gc.disable()
    
lst = [1, 2, 3]
lst.append(lst)
lst_id = id(lst)

print(lst)

del lst

gc.collect()


print("lst ref count:", PyObject.from_address(lst_id).refcnt)

gc.enable()

[1, 2, 3, [...]]
lst ref count: 0


In [54]:
len(gc.get_objects())

332630

In [55]:
import weakref

In [56]:
class A:
    def __init__(self, x):
        self.x = x

In [68]:
obj = A(10)
print("init", sys.getrefcount(obj))

weak = weakref.ref(obj)
print("weak", sys.getrefcount(obj))

obj2 = weak()
print("obj2", sys.getrefcount(obj))

print(obj is obj2)

del obj
print("del obj", sys.getrefcount(obj2))

del obj2

obj3 = weak()
print("obj3", sys.getrefcount(obj3), obj3)

init 2
weak 2
obj2 3
True
del obj 2
obj3 210328 None


In [72]:
def print_finale(*args, **kwargs):
    print("FINALE", args, kwargs)


obj = A(10)
weakref.finalize(obj, print_finale, 123)

del obj


FINALE (123,) {}


In [80]:
class Engine:
    def __init__(self, car):
        self.car = weakref.ref(car)


class Car:
    def __init__(self):
        self.engine = Engine(self)
        

my_car = Car()
del my_car

In [78]:
import cProfile, pstats, io
import time

In [82]:
pr = cProfile.Profile()
pr.enable()


def run(n):
    for i in range(n):
        time.sleep(0.1 * i)
    
    lst = [A(i) for i in range(n)]
    cars = {i: Car() for i in range(n)}
    

run(10)
    

pr.disable()


s = io.StringIO()
sortby = "cumulative"
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()

print(s.getvalue())

         99 function calls in 4.532 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        3    0.000    0.000    4.532    1.511 /Users/g.kandaurov/projects/made_advance_python_autumn_2022/.venv/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3342(run_code)
        3    0.000    0.000    4.532    1.511 {built-in method builtins.exec}
        1    0.000    0.000    4.532    4.532 /var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_63384/637529862.py:1(<module>)
        1    0.000    0.000    4.532    4.532 /var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_63384/637529862.py:5(run)
       10    4.531    0.453    4.531    0.453 {built-in method time.sleep}
        3    0.000    0.000    0.000    0.000 /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/codeop.py:149(__call__)
        3    0.000    0.000    0.000    0.000 {built-in method builtins.compile}
        1    0.000    0.0

In [84]:
!pip install memory_profiler

  Using cached memory_profiler-0.60.0.tar.gz (38 kB)
Using legacy 'setup.py install' for memory-profiler, since package 'wheel' is not installed.
    Running setup.py install for memory-profiler ... done
You should consider upgrading via the '/Users/g.kandaurov/projects/made_advance_python_autumn_2022/.venv/bin/python3 -m pip install --upgrade pip' command.


In [85]:
from memory_profiler import profile


@profile
def run(n):
    for i in range(n):
        time.sleep(0.1 * i)
    
    lst = [A(i) for i in range(n)]
    cars = {i: Car() for i in range(n)}
    

run(10)

ERROR: Could not find file /var/folders/c1/9wdb7dps2x332c6l1y2hrqkh0000gp/T/ipykernel_63384/1946970862.py


In [102]:
N = 10_000_000

class A:
    def __init__(self, x): 
        self.x = x 


class Slots:
    __slots__ = ("x",)

    def __init__(self, x): 
        self.x = x

In [103]:
%%time

lst_a = [A(i) for i in range(N)]

CPU times: user 7.24 s, sys: 651 ms, total: 7.9 s
Wall time: 9.18 s


In [104]:
%%time

lst_slot = [Slots(i) for i in range(N)]

CPU times: user 7.14 s, sys: 293 ms, total: 7.43 s
Wall time: 8.09 s


In [105]:
%%time

for obj in lst_a:
    y = obj.x
    obj.x += 10
    obj.x = 99

CPU times: user 2.11 s, sys: 210 ms, total: 2.32 s
Wall time: 2.85 s


In [106]:
%%time

for obj in lst_slot:
    y = obj.x
    obj.x += 10
    obj.x = 99

CPU times: user 1.89 s, sys: 22.8 ms, total: 1.92 s
Wall time: 2.12 s


In [ ]:
@deco_profile
def calc():
    pass


calc()
calc()
calc()
calc()


calc.print_stat()